### Setup

In [18]:
# ----------------------- #
# NOTEBOOK MPI EXPERIMENT #
# AUTHOR: XIAOYANG SONG   #
# ----------------------- #
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
from tabulate import tabulate
sys.path.append('../')
from mpi import *

### MPI Experiment

In [20]:
%%capture
from mpi import *

dir_path = "../Dataset/"
log_dir, ckpt_dir = "../checkpoint/log/", "../checkpoint/mpis/"
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-large-cased")
# version = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(version)
# model = BertLMHeadModel.from_pretrained(version)
# model_config = dict(
#     model=model,
#     tokenizer=tokenizer,
#     desc={'family': 'BERT', 'version': version}
# )

#### BERT-base

In [24]:
dset_lst = ['ocean_15']
# ALGO
algo_config = {'ll_type':'ans_inv_perp'}
# MODEL
version = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(version)
model = BertLMHeadModel.from_pretrained(version)
model_config = dict(
    model=model,
    tokenizer=tokenizer,
    desc={'family': 'BERT', 'version': version}
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
choice_lst = ['choice-only', 'desc-only', 'choice-desc']
for dset in dset_lst:
    path = dir_path + f"{dset}.csv"
    dset_config=dict(path_to_dset=path, start_idx=None, end_idx=None)
    for choice in choice_lst:
        template_config=dict(prompt = MPI_PROMPT,option=CHOICE[choice],choice=CHOICE[choice],shuffle=False)
        filename = log_fname(dset, model_config['desc'], choice, algo_config['ll_type'])
        print(filename)
        mpi = run_mpi(dset_config, model_config, algo_config, template_config, log_dir+f'{filename}.txt', False)
        torch.save(mpi, ckpt_dir+f"{filename}.pt")

#### BERT-base shuffling order MC

In [35]:
dir_path = "../Dataset/"
log_dir, ckpt_dir = "../checkpoint/log/BERT-shuffle/", "../checkpoint/mpis/BERT-shuffle/"
dset_lst = ['ocean_15']
# ALGO
algo_config = {'ll_type':'ans_inv_perp'}
# MODEL
version = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(version)
model = BertLMHeadModel.from_pretrained(version)
model_config = dict(
    model=model,
    tokenizer=tokenizer,
    desc={'family': 'BERT', 'version': version}
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
choice_lst = ['choice-only', 'desc-only', 'choice-desc']
num_mc = 5
for dset in dset_lst:
    path = dir_path + f"{dset}.csv"
    dset_config=dict(path_to_dset=path, start_idx=None, end_idx=None)
    for choice in choice_lst:
        # SHUFFLE: ORDER SYMMETRY
        for idx in range(num_mc):
            option = {'+':CHOICE[choice], '-':CHOICE[choice]}
            mpi_choice = {'+':CHOICE[choice], '-':CHOICE[choice]}
            template_config=dict(prompt = MPI_PROMPT,option=option,choice=mpi_choice,shuffle=True)
            filename = log_fname(dset, model_config['desc'], choice, algo_config['ll_type'])
            filename = filename + f"_[mc{idx}]"
            mpi = run_mpi(dset_config, model_config, algo_config, template_config, log_dir+f'{filename}.txt', False)
            torch.save(mpi, ckpt_dir+f"{filename}.pt")

100%|██████████| 15/15 [00:19<00:00,  1.30s/it]


### RoBERTa Model

In [19]:
from transformers import AutoTokenizer, RobertaModel
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
ic(outputs.last_hidden_state.shape)

ic| outputs.last_hidden_state.shape: torch.Size([1, 8, 768])


torch.Size([1, 8, 768])

In [22]:
from transformers import AutoTokenizer, RobertaForCausalLM, AutoConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
config = AutoConfig.from_pretrained("roberta-base")
config.is_decoder = True
model = RobertaForCausalLM.from_pretrained("roberta-base", config=config)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.logits

In [24]:
prediction_logits.shape

torch.Size([1, 8, 50265])